# Tutorial 3: Setting up and managing phenopype projects


A central aspect of efficiently using Phenopype is to work from within a `project`. Phenopype projects are composed of a directory tree in which each folder contains the copy or a link to a single raw image file (1). Alongside the images to be processed, users can store configuration file for the `pype` routine that we covered in the  [previous tutorial](tutorial_2_phenopype_workflow.ipynb): configurations files can be created using preconfigured templates, which can easily be modifed (2). Once raw images have been added and configuration files are in place, the `pype` routine is used until results are satisfactory (3). Together, the raw images, the `pype`-configuration files and the saved results files are all that is needed to completely reproduce any phenotypic data that was collected in the process (4). 

## Create project and add images

<center>
<div style="width:500px; text-align: left" >
    
![Step 1](_assets/workflow_high_step1.png)
    
**Step. 1**: Create a phenopype project and organize raw images into separate folders where all relevant data, attributes and results are stored. 
    
</div>
</center>

A Phenopype project directory can be initiatlized with the `project` function. The *phenoype project root folder* should be separate from the raw data, e.g. as a folder inside of your *main project folder*:

In [1]:
import phenopype as pp
import os

In [2]:
myproj = pp.project(root_dir=r"../_temp/my_project/phenopype_analysis") ## doesn't have to be "myproj", can be named anything

--------------------------------------------
phenopype will create a new project at:

../_temp/my_project/phenopype_analysis
Proceed? (y/n)
y
Warning - project root_dir already exists - overwrite? (y/n)y

"../_temp/my_project/phenopype_analysis" created (overwritten)

project attributes written to ../_temp/my_project/phenopype_analysis\attributes.yaml
--------------------------------------------


Next step is to add images to the project. You can do so with the `add_files` method of the created project (a *method* is an executable function that belongs to an existing *object*, in this case "myproj. see [this SO question](https://stackoverflow.com/questions/155609/whats-the-difference-between-a-method-and-a-function)). The function offers some flexibility in terms of which files to import. Most important arguments here are `include`, `exclude` and `filetypes`. For example, given the following list of images:

In [3]:
os.listdir("images") ## called from phenopype-master

['isopods.jpg',
 'phytoplankton.jpg',
 'snail.jpg',
 'stickle1.JPG',
 'stickle2.JPG',
 'stickle3.JPG',
 'stickleback_side.jpg',
 'stickleback_top.jpg']

If we want to import "stickle1", "stickle2", and "stickle3", we can do a combination `include` and `exclude` (also prints all other default settings):

In [4]:
myproj.add_files(image_dir="images",
                 include="stickle",       ## can be type "str" or type "list"
                 exclude=["side","top"]   ## can be type "str" or type "list"
                ) 

--------------------------------------------
phenopype will search for files at

images

using the following settings:

filetypes: ['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png'], include: stickle, exclude: ['side', 'top'], raw_mode: copy, search_mode: dir, unique_mode: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created

Found 3 files
--------------------------------------------


The three images have the same (nonstandard) file ending, so we can also use the filetype argument (and the overwrite argument, because have already added them above):

In [5]:
 myproj.add_files(image_dir="images",
                 filetypes="JPG" ,      ## can be type "str" or type "list"
                 overwrite=True
                ) 

--------------------------------------------
phenopype will search for files at

images

using the following settings:

filetypes: JPG, include: [], exclude: [], raw_mode: copy, search_mode: dir, unique_mode: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created (overwritten)
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created (overwritten)
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created (overwritten)

Found 3 files
--------------------------------------------


The remaining settings are `raw_mode`, `search_mode`, `unique_mode`. `raw_mode` determines whether raw files should be copied to each folder in the Phenopype directory tree (using `copy` [default]), or just their filepath (using `link`), which can be useful if data sets contain many or very large images. `search_mode` indicates whether only the top directory (`dir` [default]), or also all subdirectories (`recursive`) should be included in the search. `unique_mode` indicates whether files should be unique by their path (`filepath` [default]) or only by their name (`filename`) - duplicate files will be skipped.  

In [6]:
help(pp.project.add_files)

Help on function add_files in module phenopype.main:

add_files(self, image_dir, filetypes=['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png'], include=[], exclude=[], raw_mode='copy', search_mode='dir', unique_mode='path', overwrite=False, resize=1, **kwargs)
    Add files to your project from a directory, can look recursively. 
    Specify in- or exclude arguments, filetypes, duplicate-action and copy 
    or link raw files to save memory on the harddrive. For each found image,
    a folder will be created in the "data" folder within the projects root
    directory. If found images are in subfolders and search_mode is 
    recursive, the respective phenopype directories will be created with 
    flattened path as prefix. 
    
    E.g., with "raw_files" as folder with the original image files 
    and "phenopype_proj" as rootfolder:
    
    - raw_files/file.jpg ==> phenopype_proj/data/file.jpg
    - raw_files/subdir1/file.jpg ==> phenopype_proj/data/1__subdir1__file.jpg
    - raw_files/sub

## Add `pype`-configuration files 

<center>
<div style="width:500px; text-align: left" >
    
![Step 2](_assets/workflow_high_step2.png)
    
**Step. 2**: Create configuration files and store them alongside the raw images.
    
</div>
</center>

In the next step we prepare the files we added for use with the `pype` routine by addding a configuration file with the `add_config` method. Instead of adding the functions one by one we can load presets that are appropriate for the given computer vision analysis.

Currently, the different templates are stored inside a Python file, and can be inspected using `dir(pp.presets)` to show all existing presets, and `print(pp.presets.landmarks_plain)` to show the contents.

In [7]:
dir(pp.presets)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'demo1',
 'inverted1',
 'landmarks_2',
 'landmarks_plain',
 'object_detection_1',
 'preset1',
 'preset3',
 'preset4']

In [8]:
print(pp.presets.landmarks_plain)


measurement:
- landmarks:
    point_size: 25
    point_col: green
    label_size: 3
    label_width: 5
visualization:
- show_landmarks:
    point_size: 25
    point_col: green
    label_size: 3
    label_width: 5
export:
- save_landmarks



For example, if we want to place landmarks, we can use one of the corresponding presets.  


In [9]:
myproj.add_config(name = "lm", config_preset="landmarks_plain")

pype config generated from landmarks_plain.
pype_lm.yaml created for 0__stickle1
pype_lm.yaml created for 0__stickle2
pype_lm.yaml created for 0__stickle3


Now all images folders contain a configuration file in *yaml*  format (see [Tutorial 2](tutorial_2_phenopype_workflow.ipynb#yaml-syntax) and the [resources section](resources.html) of the Documentation for details). 

An imporant feature of `add_config` is the opportunity to evulate and edit the template before it gets saved in the folders. This is done by setting the flag `interactive=True` in the arguments. For example, if we globally want to change point and label size of the landmark preset, we can do: 

<center>
<div style="width:500px; text-align: left" >
    
![Edit template](_assets/change_template.png)
    
Edit the templates before saving them to the image folders.
    
</div>
</center>

**NOTE 1:** The `pype` function opens a text editor and a python window. To modify the `pype` configuration templates, by default, the first image in your project directory tree will copied over to the phenopype project root directory. After the windows have opened they can be controlled as described as in [Tutorial 2](tutorial_2_phenopype_workflow.ipynb#pype-behavior).

**NOTE 2:** If you have issues with this step, e.g. no text editor window is popping up, make sure you have set the default app for opening *yaml* files. Furthermore, consult the [installation instructions](installation.html#choose-a-text-editor) and check if your text editor is configured correctly. 


In [10]:
myproj.add_config(name = "lm", 
                  config_preset="landmarks_plain",
                  interactive=True,
                  overwrite=True                 ## needed because config with the name "lm" already exists in the folders
                 )

pype config generated from landmarks_plain.


------------+++ new pype iteration 2020:03:31 13:45:14 +++--------------


MEASUREMENT
landmarks
- setting landmarks
- terminated polyline creation
RESTART


------------+++ new pype iteration 2020:03:31 13:45:20 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks


TERMINATE
pype_lm.yaml created for 0__stickle1 (overwritten)
pype_lm.yaml created for 0__stickle2 (overwritten)
pype_lm.yaml created for 0__stickle3 (overwritten)


In [11]:
help(pp.project.add_config)

Help on function add_config in module phenopype.main:

add_config(self, name, config_preset='preset1', interactive=False, overwrite=False, **kwargs)
    Add pype configuration presets to all image folders in the project, either by using
    the templates included in the presets folder, or by adding your own templates
    by providing a path to a yaml file. Can be tested and modified using the 
    interactive flag before distributing the config files.
    
    Parameters
    ----------
    
    name: str
        name of config-file. this gets appended to all files and serves as and
        identifier of a specific analysis pipeline
    preset: str, optional
        can be either a string denoting a template name (e.g. preset1, preset2, 
        landamarking1, ... - in "phenopype/settings/presets.py") or a path to a 
        compatible yaml file
    interactive: bool, optional
        start a pype and modify preset before saving it to phenopype directories
    overwrite: bool, optional


## Saving and loading a project

Project objects can be saved using a the static method `save` (static = unbound to any object). This will save the project data to the project's root directory. Currently, the only useful information stored in the project object is the list of all contained directories. Future releases will make more use of the project object. 

**NOTE:** `pp.project.save` saves ONLY the project data, all data collected with the pype method or any of the other workflows need to be saved inside the folders using the appropriate [export](api.html#export) functions.

In [12]:
pp.project.save(myproj, overwrite=True)

Project data saved under ../_temp/my_project/phenopype_analysis\project.data.


To load the project again, add provide the path of the `project.data` file in the root folder to the `load` method:

In [13]:
myproj = pp.project.load(r"../_temp/my_project/phenopype_analysis/project.data")

## Applying the pype to project folders

<center>
<div style="width:800px; text-align: left" >
    
![Step 3](_assets/workflow_high_step3.png)
    
**Step. 3**: Apply `pype` function image by image.
    
</div>
</center>

After adding images and configuration, all is set to process your dataset with high throughput. Using a simple `for` loop, we go through all directories one by one. You can modify the configuration file and controll the window as described as in [Tutorial 2](tutorial_2_phenopype_workflow.ipynb#yaml-syntax). The `skip` argument will allow to skip files with a given config name you have already analyzed. This allows you to return to the point where you left off.

**NOTE 1:** Make sure to specifiy the name of the config file you added before, in this case, "lm". The config file name serves multiple purposes: on the one hand it tells the `pype` function which configuration to load, if you have multiple in one directory. On the other hand, the name gets appended to all results files that are produced by this constellation.  

**NOTE 2:** Consult [Tutorial 2](tutorial_2_phenopype_workflow.ipynb#pype-behavior) to understand `pype` behavior. For example, the `pype` will automatically save all collected data, and by default overwrite any existing results files, but the latter only if indicated in the config file.   

In [14]:
for img in myproj.dirpaths:
    pp.pype(img, 
            name="lm",         ## loads the config file "pype_config_lm.yaml". "lm" gets appended to all results files
            skip=True          ## skip=True will skip over any directories that already contain results files with "lm"
           )



------------+++ new pype iteration 2020:03:31 13:45:35 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_project/phenopype_analysis\data\0__stickle1\landmarks_lm.csv.
AUTOSAVE
save_canvas
- canvas saved under ../_temp/my_project/phenopype_analysis\data\0__stickle1\canvas_lm.jpg.


TERMINATE


------------+++ new pype iteration 2020:03:31 13:45:42 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_project/phenopype_analysis\data\0__stickle2\landmarks_lm.csv.
AUTOSAVE
save_canvas
- canvas saved under ../_temp/my_project/phenopype_analysis\data\0__stickle2\canvas_lm.jpg.


TERMINATE


------------+++ new pype iteration 2020:03:31 13:45:46 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_project/

<center>
<div style="width:500px; text-align: left" >
    
![Step 4](_assets/workflow_high_step4.png)
    
**Step. 4**: Each folder contains all information necessary to reproduce the collected phenopytic data. Ouput from different `pype` runs can be stored side by side in the same folders. 
    
</div>
</center>

As mentioned above, it's possible to have multiple configuration files side by side in phenopype folders. For example, if we want to implement an alternative set of landmarks, we can simply do:

In [15]:
myproj.add_config(name = "lm2",                  ## add different name (my not contain underscores or other special characters)
                  config_preset="landmarks_plain"    ## same preset
                 )

pype config generated from landmarks_plain.
pype_lm2.yaml created for 0__stickle1
pype_lm2.yaml created for 0__stickle2
pype_lm2.yaml created for 0__stickle3


In [16]:
for img in myproj.dirpaths:
    pp.pype(img, 
            name="lm2",         ## loads the config file "pype_config_lm2.yaml". "lm2" gets appended to all results files
            skip=True          ## skip=True will skip over any directories that already contain results files with "lm2"
           )



------------+++ new pype iteration 2020:03:31 13:45:58 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_project/phenopype_analysis\data\0__stickle1\landmarks_lm2.csv.
AUTOSAVE
save_canvas
- canvas saved under ../_temp/my_project/phenopype_analysis\data\0__stickle1\canvas_lm2.jpg.


TERMINATE


------------+++ new pype iteration 2020:03:31 13:46:00 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_project/phenopype_analysis\data\0__stickle2\landmarks_lm2.csv.
AUTOSAVE
save_canvas
- canvas saved under ../_temp/my_project/phenopype_analysis\data\0__stickle2\canvas_lm2.jpg.


TERMINATE


------------+++ new pype iteration 2020:03:31 13:46:03 +++--------------


MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
show_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/my_proj

In [17]:
os.listdir(r"../_temp/my_project/phenopype_analysis\data\0__stickle1")

['attributes.yaml',
 'canvas_lm.jpg',
 'canvas_lm2.jpg',
 'landmarks_lm.csv',
 'landmarks_lm2.csv',
 'pype_config_lm.yaml',
 'pype_config_lm2.yaml',
 'raw.JPG']